In [7]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()

# This shows the mean, standard deviation, minimum, and other metrics for our numeric columns.
df.describe()

# Get some data on our boolean column indicating the baby's gender.
df['is_male'].value_counts()

True     5158
False    4842
Name: is_male, dtype: int64

In [8]:
# Drop rows with null values from the dataset and shuffle the data.
df = df.dropna()
df = shuffle(df, random_state=2)

# Extract the label column into a separate variable and create a DataFrame with only our features.
# Since is_male is a boolean, we'll convert it to an integer so that all inputs to our model are numeric.
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
757,7.374463,True,23,1,39.0
6544,8.375361,True,19,1,38.0
3190,6.239082,True,28,1,40.0
2064,8.437091,False,32,1,40.0
4066,7.374463,False,17,1,40.0


In [ ]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)